In [1]:
import sys
print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [2]:
# Compute Precision, Recall, and F1-score
def get_prf1_score(y_true, y_pred):
    tp, fp, fn = 0.0, 0.0, 0.0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            tp += 1
        elif y_pred[i] > y_true[i]:
            fp += 1
        else:
            fn += 1
    if tp == 0:
        tp = 1.0
    if fp == 0:
        fp = 1.0
    if fn == 0:
        fn  = 1.0
    P = tp / (tp + fp)
    R = tp / (tp + fn)
    F = 2 * P * R / (P + R)
    print ('\nPrecision: {0}\t Recall: {1}\t F1-Score: {2}'.format(P, R, F))
    return {'P': P, 'R': R, 'F': F}

###new added file
def oe_score(y_true, y_pred):
    oe_no=0
    nt=len(y_pred)
    for i in range(nt):
        if abs(y_pred[i]-y_true[i])>1:
            oe_no+=1
    OE= oe_no/nt
    print('OE:{}'.format(OE))
    return {'OE': OE}    

### Test the precision, Recall, F1-score, OE score

In [3]:
y_true,y_pred=[0,1,2,2,4,3],[0,1,4,2,3,2]
get_prf1_score(y_true, y_pred),oe_score(y_true, y_pred)


Precision: 0.75	 Recall: 0.6	 F1-Score: 0.6666666666666665
OE:0.16666666666666666


({'P': 0.75, 'R': 0.6, 'F': 0.6666666666666665}, {'OE': 0.16666666666666666})

In [4]:
###unzip the file
import gzip
import shutil
import os

path=r'C:\GAOMSA_Comb\OMSA_Summer_2021_and_later\Spring2022\CSE6250\BHDProject\Project\Suicide-Severity\conceptnet-numberbatch-master'
os.chdir(path)

import gzip
import shutil
f_in=open
with gzip.open('numberbatch-en-19.08.txt.gz', 'rb') as f_in:
    with open('numberbatch-en.txt','wb') as f_out:
        shutil.copyfileobj(f_in,f_out)

In [5]:
###test vectorize_data()
import csv
import string
from nltk import word_tokenize
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from keras.utils.np_utils import to_categorical
import datetime, time
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input, MaxPool2D
from keras.layers import Conv2D, GlobalAveragePooling1D, MaxPooling2D
from keras.layers import Concatenate
# from keras.optimizers import Adam
# https://stackoverflow.com/questions/62707558/importerror-cannot-import-name-adam-from-keras-optimizers
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

In [6]:
sys_params = {'emb_dim': 300,
              'max_sent_len': 1500,
              'str_padd': '@PADD',
              'cross_val': 5}
w2v_file = {'file': 'numberbatch-en.txt', 'is_binary': False}

In [7]:
# Vectorize the input data using pretrained word2vec embedding lookup
def vectorize_data(lst_input):

    padd = sys_params['str_padd']
    wv_size = sys_params['emb_dim']

    # Load the pre-trained word2vec model
    w2v_model = KeyedVectors.load_word2vec_format(w2v_file['file'], binary=w2v_file['is_binary'])

    # Get the word2vec vocabulary
    # vocab = w2v_model.vocab  ##out of date
    vocab =w2v_model.key_to_index
    #
    padding_zeros = np.zeros(wv_size, dtype=np.float32)

    x_data = []

    # Loop through each sentence
    for sent in lst_input:
        emb = []
        for tok in sent:

            # # Zero-padding for padded tokens
            # if tok == padd:
            #     emb.append(list(padding_zeros))

            # # Get the token embedding from the word2vec model
            # elif tok in vocab.keys():
            #     emb.append(w2v_model[tok].astype(float).tolist())
            # Zero-padding for padded tokens
            if tok.lower() == padd:
                emb.append(list(padding_zeros))

            # Get the token embedding from the word2vec model
            elif tok.lower() in vocab.keys():
                emb.append(w2v_model[tok.lower()].astype(float).tolist())
            # Zero-padding for out-of-vocab tokens
            else:
                emb.append(list(padding_zeros))

        x_data.append(emb)

    del w2v_model, vocab

    return np.array(x_data)

### Make sure the lst_input format to be a list.

In [8]:
###example to use vectorize_data()
vectorize_data([["I","am","a","robot"]])

array([[[-0.1129    , -0.14210001,  0.0169    , ...,  0.0227    ,
          0.0246    ,  0.0084    ],
        [-0.0615    , -0.1116    , -0.03      , ..., -0.0007    ,
          0.0232    , -0.0141    ],
        [-0.1011    , -0.0806    , -0.0092    , ...,  0.0398    ,
         -0.0122    , -0.0257    ],
        [-0.0715    ,  0.0022    ,  0.0066    , ...,  0.0061    ,
          0.0114    , -0.0099    ]]])